In [ ]:
pip install tensorflow==1.15


     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 3.8MB 53.7MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 512kB 54.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7529e76f5f40b06922894bc52c073aa3cc912b9ec649f502640b7403c16f1f6c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [ ]:
d = 3
if d == 1 :
  !git clone https://github.com/Skuldur/Classical-Piano-Composer
  files_path ='/content/Classical-Piano-Composer/midi_songs'
elif d == 2 :
  !wget https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
  !unzip maestro-v2.0.0-midi.zip
  files_path = '/content/maestro-v2.0.0/2018'
else:
  !git clone https://github.com/Simeonedef/music_gen_deep_learning
  files_path = '/content/music_gen_deep_learning/midi/'


import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
from __future__ import print_function, division
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
from tensorflow.python.compiler.tensorrt import trt_convert as trt

def get_notes():
    """ Get all the notes and chords from the midi files """
    files = glob.glob(files_path+'/*.mid*')
    note_list = []
    try :
        F = open('NoteList','rb')
        note_list = pickle.load(F)
        F.close
        print("List de notes Trouvée et chargé")
        print("le nombre total de son est : ",len(note_list))
    except : 
        n_chords = 0
        n_notes = 0
        n_silences = 0
        i = 0
        for f in files :
            print("\rprocessing","."*((i+1)%4),end='')
            song = converter.parse(f)
            #on verifie si il y'a plusieur instrument si c'est le cas on prend que la partie piano
            instrument_parts = instrument.partitionByInstrument(song)
            if instrument_parts :
                song_notes = instrument_parts.parts[0].recurse()
            else:
                song_notes = song.flat.notes
            # on boucle sur les notes et on les ajoutes a note_liste
            for n in song_notes : 
                if isinstance(n,note.Note):
                    n_notes += 1 
                    note_list.append(str(n.pitch))
                elif isinstance(n,note.Rest):
                    n_silences += 1
                    note_list.append('SC')
                elif isinstance(n,chord.Chord):
                    n_chords += 1
                    chrd = '.'.join(str(x) for x in n.normalOrder)
                    note_list.append(chrd)
            i += 1
        if i!=0:
          print("\nDone!")
          print("le nombre de morceau est : ",len(files))
          print("le nombre total de son est : ",len(note_list))
          print("le nombre total de note est : ",n_notes)
          print("le nombre total d'accords est : ",n_chords)
          print("le nombre total de silence est : ",n_silences)
          with open('NoteList','wb') as F:
              pickle.dump(note_list,F)
          print('notes sauvgarder dans le fichier : ','NoteList')
        else :
          print('no notes founds')

    return note_list


def prepare_sequences(notes):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    n_vocab=len(set(notes))

    # Get all possible notes (pitch, chords, SC)
    noteNames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(noteNames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length] #the next note in the seq
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input) #how many seq do we have 

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # Normalize input between -1 and 1
    network_input = (network_input - float(n_vocab)/2) / (float(n_vocab)/2)
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def generate_notes(model, network_input):
    """ Generate notes from the neural network based on a sequence of notes """
    notes = get_notes()
    n_vocab=len(set(notes))
    

    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    # Get pitch names and store in a dictionary
    noteNames = sorted(set(item for item in notes))
    int_to_note = dict((number, note) for number, note in enumerate(noteNames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        #prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        #ind = np.unravel_index(np.argmax(a, axis=None), a.shape) result=predection[ind] !
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = numpy.append(pattern,index)
        #pattern.append(index)
        pattern = pattern[1:len(pattern)] #len(pattern)-1 ?

    return prediction_output

def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    notes = get_notes()
    noteNames = sorted(set(item for item in notes))
    

    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for item in prediction_output:
        pattern = item
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a SC or note
        else :
          if pattern != 'SC' and pattern in noteNames :
            new_note = note.Note(pattern)
            new_note.storedInstrument = instrument.Piano()
          else :
            new_note = note.Rest()

          new_note.offset = offset
          output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

class GAN():
    def __init__(self, rows):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        self.notes = get_notes()
        self.n_vocab=len(set(self.notes))
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_discriminator(self):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=self.seq_shape, return_sequences=True)) #faster then LSTM 
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid')) #binary_classification
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

         model = Sequential()
         model.add(Dense(256, input_dim=self.latent_dim))
         model.add(LeakyReLU(alpha=0.2))
         model.add(BatchNormalization(momentum=0.8))
         model.add(Dense(512))
         model.add(LeakyReLU(alpha=0.2))
         model.add(BatchNormalization(momentum=0.8))
         model.add(Dense(1024))
         model.add(LeakyReLU(alpha=0.2))
         model.add(BatchNormalization(momentum=0.8))
         model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
         model.add(Reshape(self.seq_shape))
         model.summary()
        #model.add(Embedding(vocab_len,embedding_dim,batch_input_shape=[batch_size, None]))
        #model.add(LSTM(512,return_sequences=True,stateful=True))
        #model.add(Dropout(0.2))
        #model.add(TimeDistributed(Dense(vocab_len,activation='softmax')))
        #model.summary()
         noise = Input(shape=(self.latent_dim,))
         seq = model(noise)

         return Model(noise, seq)

    def train(self, epochs, batch_size=128, sample_interval=50):
        

        X_train, y_train = prepare_sequences(self.notes)

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            self.discriminator.trainable = True
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) #la moyenne of the lost fuction of the discriminator


            # Train the generator (to have the discriminator label samples as real)
            self.discriminator.trainable = False #frose the disciminator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            g_loss = self.combined.train_on_batch(noise, real) 

            # Print the progress and save into loss lists
            if epoch % sample_interval == 0:
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
              self.disc_loss.append(d_loss[0])
              self.gen_loss.append(g_loss)
        
        
    def generate(self):
        # Get notes names and store in a dictionary
  
        noteNames = sorted(set(item for item in self.notes))
        int_to_note = dict((number, note) for number, note in enumerate(noteNames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        

        pred_notes = [((x+1)/2)*self.n_vocab for x in predictions[0]]
        pred_notes = [int_to_note[int(x)] for x in pred_notes]
        
        create_midi(pred_notes, 'gan_final')
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig('GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

if __name__ == '__main__':
  gan = GAN(rows=100)    
  gan.train(epochs=10, batch_size=32, sample_interval=1)
  gan.plot_loss()


Cloning into 'music_gen_deep_learning'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 200 (delta 4), reused 7 (delta 2), pack-reused 191
Receiving objects: 100% (200/200), 39.67 MiB | 37.79 MiB/s, done.
Resolving deltas: 100% (43/43), done.


ImportError: ignored

In [ ]:
  gan.generate()